In [3]:
# Steps: 1.Data generation 2.Collision Constructor 3. Dataset Building 4. 
# A function for use to generate data and validate it in four steps 
# (1) Naive Method -> just satisfy masss conservation continiuity eq. [This model is computationaly expensive and doesn't guarantee physical constraints]
# (2) Satisfying symmetric condition byenforcing \phi_NN be equivarience in respect to D_8 using group averaging [This method doesn't satisfy Postulate 3 mass & moment invarient cond.]
# (3) Conservation of both Mass and Monmentum in x and y dir in which we need to use Algebraic fix: [This method does't satisfy Postulate 2 equivariance cond.]
# (3.1) Algebraic Reconstruction (Biased for rows 2, 5, & 8). Why?
# (3.2) Symmetric Algebraic Reconstruction with group-averaging method
# (3.3) Penalize mass and momentum mismatches with a soft constraint in the loss function
# Combined Symmetric-Conservation to satisfy all 4 Postulates at once and be computationally efficient by reducing degrees of freedom for D2Q9 from 90 down to 18

import numpy as np

def generate_training_data_bgk(
    N_samples=1200000,
    rho_min=0.5, rho_max=2.0,
    u_max=0.03,
    tau=1.0,
    c_s=1.0/np.sqrt(3.0)
):
    """
    Generates (f_pre, f_post)(N_samples, 9) for the D2Q9 BGK model by:
    We sample macro values for density & velocity for our training data to use in NN, by using f_eq.i and
    then adding perturbation to that. Finally we calculate BGK post collision for f_post.i ...
    """
    # D2Q9 discrete velocities
    V_d = np.array([
        [ 0,  0],
        [ 1,  0],
        [ 0,  1],
        [-1,  0],
        [ 0, -1],
        [ 1,  1],
        [-1,  1],
        [-1, -1],
        [ 1, -1]
    ], dtype=np.float64)

    # Standard D2Q9 weights
    w = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36], dtype=np.float64)

    f_pre_list, f_post_list = [], []  # Generate list to append later

    for _ in range(N_samples):
        # # Alocating random density & velocity
        rho, speed, angle = np.random.uniform([rho_min, 0.0, 0.0], [rho_max, u_max, 2*np.pi])
        ux = speed * np.cos(angle)
        uy = speed * np.sin(angle)

        # ## Compute f_eq.i with 2_nd order Hermite expansion
        f_eq = np.zeros(9, dtype=np.float64)
        for i in range(9):
            cu = V_d[i,0]*ux + V_d[i,1]*uy
            f_eq[i] = w[i] * rho * (
                1.0 + (cu)/(c_s**2) + 0.5*((cu)/(c_s**2))**2 - 0.5*( (ux**2 + uy**2)/(c_s**2) )
            )

        # ### Generate perturbation fneq with mass & momentum sum==0
        
        ## -------------
        ## -- Phase 1 --
        ## -------------
        raw_perturb = np.random.normal(loc=0.0, scale=0.001, size=9)
        
        # -- Mass Conservation --
        raw_perturb -= np.mean(raw_perturb) # shifts all perturbations by the average to satisfy continuity

        # The folowing is commented for use in more advanced method which needs improvement  (to be complited)
        """ 
        # -- Momentum Conservation --
        vx_all = velocities[:,0]  # array of shape (9,)
        vy_all = velocities[:,1]  

        # Current net momentum in x, y due to raw_perturb:
        dx = np.sum(raw_perturb * vx_all)
        dy = np.sum(raw_perturb * vy_all)

        # Build the 2x2 matrix for the linear system:
        # [A11 A12] [alpha] = [dx]
        # [A21 A22] [beta ]   [dy]
        # where alpha, beta are how much to project out
        A11 = np.sum(vx_all * vx_all)
        A12 = np.sum(vx_all * vy_all)
        A21 = A12
        A22 = np.sum(vy_all * vy_all)

        # Solve for alpha, beta
        #   [alpha, beta]^T = inv(A) * [dx, dy]^T
        # A = [[A11, A12],[A21,A22]]
        # b = [dx, dy]
        detA = A11*A22 - A12*A21
        if abs(detA) > 1e-14:
            alpha = ( A22*dx - A12*dy ) / detA
            beta  = (-A21*dx + A11*dy ) / detA
        else:
            # fallback if degenerate; usually won't happen in D2Q9
            alpha, beta = 0.0, 0.0

        # Subtract alpha*vx + beta*vy from raw_perturb
        raw_perturb -= alpha*vx_all + beta*vy_all
        
        ## -------------
        ## -- Phase 2 --
        ## -------------

        # (C) Zero net mass again
        raw_perturb -= np.mean(raw_perturb)

        # (D) Zero net momentum in x,y again
        dx2 = np.sum(raw_perturb * vx_all)
        dy2 = np.sum(raw_perturb * vy_all)

        if abs(detA) > 1e-14:
            alpha2 = (A22 * dx2 - A12 * dy2) / detA
            beta2  = (-A21 * dx2 + A11 * dy2) / detA
        else:
            alpha2, beta2 = 0.0, 0.0

        raw_perturb -= alpha2 * vx_all + beta2 * vy_all
        """
        
        f_pre = f_eq + raw_perturb

        # #### Compute BGK
        f_post = f_pre - (1.0/tau)*(f_pre - f_eq)

        f_pre_list.append(f_pre)
        f_post_list.append(f_post)

    f_pre_array = np.array(f_pre_list)
    f_post_array = np.array(f_post_list)

    return f_pre_array, f_post_array



In [4]:
import sys
print(sys.executable)

/home/deo/Programms/MLBM/mlenv/bin/python


In [10]:
# Constructor for handling layers for each collision

import torch
print(torch.__version__)
import torch.nn as nn
import torch.optim as optim

class NaiveCollision(nn.Module): 
    def __init__(self, hidden_size=50):
        super().__init__()
        # Naive simple 9->hidden->hidden->9 layer definition
        self.network = nn.Sequential(
            nn.Linear(9, hidden_size, bias=False),
            nn.ReLU(), # Hidden Layer activation
            nn.Linear(hidden_size, hidden_size, bias=False),
            nn.ReLU(),
            nn.Linear(hidden_size, 9, bias=False)  # final: 9 outputs
        )
    
    def forward(self, x):
        return self.network(x)

2.6.0+cu124


In [11]:
# Create MSRE loss function

# import torch
# import torch.nn as nn

class MSRELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super(MSRELoss, self).__init__()
        self.eps = eps

    def forward(self, input, target):
        relative_error = (input - target) / (target + self.eps)
        return torch.mean(relative_error ** 2)

In [14]:
# We need to build a dataset and train it 

from torch.utils.data import DataLoader, TensorDataset

def train_naive(f_pre, f_post, 
                    epochs=200, batch_size=32, lr=1e-3,  # learning rate in W := W - lr * dL/dw
                    hidden_size=50, device='cpu'):
    """
    f_pre, f_post: NumPy arrays, shape (N,9).
    We'll do naive MLP training for mapping f_pre->f_post.
    returns trained model
    """
    X = torch.from_numpy(f_pre).float().to(device)
    Y = torch.from_numpy(f_post).float().to(device)
    dataset = TensorDataset(X, Y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # Does the splitting, shuffling, and iterate 
    
    # Create Network + Optimizer + Loss Function
    network = NaiveCollision(hidden_size=hidden_size).to(device) # we have created the network on device named netwrok
    optimizer = optim.Adam(network.parameters(), lr=lr) 
    lossFunction = MSRELoss()
    
    # Main loop
    network.train() # This put the network in training mode
    for ep in range(epochs):
        total_loss = 0.0 # What we want (Our Goal)
        for batch_x, batch_y in dataloader: # batch_x == (batch_size, 9)
            
            optimizer.zero_grad() # Reset .gard from the past loop (forward -> backward -> update)
            
            pred_y = network(batch_x) # Compute the predicted y from x fed to the network on a chosen device
            loss = lossFunction(pred_y, batch_y) # MSRE
            
            loss.backward() # W := W - lr * dL/dw
            optimizer.step() # Update the weights
            total_loss += loss.item()*batch_x.size(0) # We get the float from loss * batch size 
            
        avg_loss = total_loss / len(dataset)
        if (ep+1)%10==0:
            print(f"Epoch {ep+1}/{epochs}, Loss={avg_loss:.6f}")
    return network

In [ ]:
def main_naive():
    import torch
    
    # Generate
    f_pre_train, f_post_train = generate_training_data_bgk(N_samples=1000000)
    f_pre_test, f_post_test = generate_training_data_bgk(N_samples=200000)
    
    # Train
    device="cpu"
    model = train_naive(f_pre_train, f_post_train,
                            epochs=200, 
                            batch_size=32,
                            lr=1e-3,
                            hidden_size=50,
                            device="cpu")
    
    # Evaluate on test
    model.eval()
    with torch.no_grad():
        x_t = torch.from_numpy(f_pre_test).float().to(device)
        y_true = torch.from_numpy(f_post_test).float().to(device)
        y_pred = model(x_t)
        mse_test = torch.mean((y_pred - y_true)**2).item()
    print(f"Test MSE = {mse_test:.6e}")


if __name__=="__main__":
    main_naive()


Epoch 10/200, Loss=0.000051
Epoch 20/200, Loss=0.000050
Epoch 30/200, Loss=0.000049
Epoch 40/200, Loss=0.000049
Epoch 50/200, Loss=0.000049
